# Naive Bayes on the Emoji data

In [1]:
import random
import twitter
import emoji
# import itertools
import pandas as pd
from itertools import chain
import numpy as np
#from sklearn.model_selection import train_test_split
import os
# import nb_utils


In [2]:
all_tweets = pd.read_csv('data/emojis_homemade.csv')
all_tweets['emoji'].value_counts()

/home/nickdbn/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


😂        50801
😭        20974
❤        17476
emoji    12017
😍        11528
🔥        11202
🤣         7117
🙏         5739
😩         5595
🤔         5538
💕         5449
🖤         5030
😊         5020
✨         4455
👀         4425
💜         4266
👏         4168
💀         3953
🙄         3939
🎉         3676
🙌         3643
😘         3574
💯         3279
😔         3071
😏         3047
💙         3046
👍         2943
♥         2833
😉         2730
👇         2710
         ...  
👝            1
🏣            1
🛤            1
🚈            1
🚄            1
🧙            1
🧘            1
🔕            1
🤵            1
◀            1
㊗            1
🤼            1
📗            1
⚱            1
👲            1
🚟            1
🈵            1
👷            1
🚇            1
🈳            1
🉐            1
🕍            1
⛎            1
🕠            1
🗃            1
➗            1
🕕            1
📏            1
🀄            1
🥫            1
Name: emoji, Length: 1074, dtype: int64

In [3]:
all_tweets.shape

(576506, 3)

In [8]:
tweets = all_tweets.groupby('emoji').filter(lambda c:len(c) > 1000)
print ("tweets shape:", tweets.shape)
tweets['emoji'].value_counts()

tweets shape: (312679, 3)


😂        50801
😭        20974
❤        17476
emoji    12017
😍        11528
🔥        11202
🤣         7117
🙏         5739
😩         5595
🤔         5538
💕         5449
🖤         5030
😊         5020
✨         4455
👀         4425
💜         4266
👏         4168
💀         3953
🙄         3939
🎉         3676
🙌         3643
😘         3574
💯         3279
😔         3071
😏         3047
💙         3046
👍         2943
♥         2833
😉         2730
👇         2710
         ...  
😌         1656
💥         1654
💔         1648
✌         1603
😫         1592
🤝         1579
👉         1561
🙂         1538
😈         1524
💚         1495
☹         1475
➡         1458
🤩         1442
🎶         1441
✅         1379
😋         1368
😆         1353
💞         1313
💓         1309
👑         1215
▶         1193
😡         1187
😪         1160
🏆         1112
😀         1111
😬         1088
😇         1051
📷         1048
🤤         1009
💋         1006
Name: emoji, Length: 83, dtype: int64

In [5]:
max(tweets['text'], key=lambda t:len(t))

'RT @JoanKareri: A guy who smells good and leaves you with his scent after he leaves&gt;&gt;&gt;&gt; &gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;'

In [6]:
chars = list(sorted(set(chain(*tweets['text']))))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
max_sequence_len = max(len(x) for x in tweets['text'])

emojis = list(sorted(set(tweets['emoji'])))
emoji_to_idx = {em: idx for idx, em in enumerate(emojis)}
emojis[:10]

['emoji', '©', '‼', '▶', '☹', '☺', '♥', '✅', '✌', '✨']

In [10]:
#train_tweets = tweets[0:10000]
#dev_tweets = tweets[10000:20000]
#test_tweets = tweets[20000:30000]


# !! Temp! Try on much bigger dataset
#train_tweets = tweets[0:100000] # 100000 tweets
#dev_tweets = tweets[100000:110000]
#test_tweets = tweets[110000:120000]

# All the data
train_tweets = tweets[0:-20000] # 100000 tweets
dev_tweets = tweets[-20000:-10000]
test_tweets = tweets[-10000:-1]


In [11]:
import sklearn
import numpy as np
import collections 

In [12]:
from sklearn import feature_extraction, preprocessing, naive_bayes, metrics

In [13]:
vocab_size = 10000

tfidf_vec = feature_extraction.text.TfidfVectorizer(max_features=vocab_size)
label_encoder = preprocessing.LabelEncoder()
train_x = tfidf_vec.fit_transform(train_tweets['text'])
train_y = label_encoder.fit_transform(train_tweets['emoji'])
dev_x = tfidf_vec.fit_transform(dev_tweets['text'])
dev_y = label_encoder.fit_transform(dev_tweets['emoji'])

In [14]:
bayes = naive_bayes.MultinomialNB()
bayes.fit (train_x, train_y)
pred = bayes.predict (dev_x)
metrics.precision_score(pred, dev_y, average='micro')

0.0409

In [15]:
d = np.eye (len(tfidf_vec.vocabulary_))
word_pred = bayes.predict_proba(d)

In [16]:
tfidf_vec.vocabulary_

{'rt': 7186,
 'joeysofficial': 3972,
 've': 9208,
 'watched': 9378,
 'this': 8908,
 'video': 9232,
 'about': 387,
 '20': 71,
 'times': 8950,
 'now': 4910,
 'and': 775,
 'it': 3866,
 'has': 3498,
 'made': 4451,
 'me': 4553,
 'smile': 8262,
 'more': 4709,
 'each': 2801,
 'time': 8948,
 'https': 3654,
 'co': 2312,
 'ykotdvmfov': 9856,
 'get': 3302,
 'well': 9481,
 'soon': 8406,
 'ate': 1165,
 'fighting': 3089,
 'said': 7343,
 'marty': 4511,
 'big': 1844,
 'one': 5030,
 'so': 8320,
 'does': 2698,
 'coffee': 2320,
 'just': 4016,
 'slip': 8222,
 'have': 3508,
 'no': 4884,
 'knowledge': 4128,
 'either': 2841,
 'way': 9405,
 'oneinanarmy': 5034,
 'army': 1024,
 'the': 8863,
 'next': 4848,
 'set': 7740,
 'of': 4951,
 'you': 9864,
 'provided': 6165,
 'went': 9493,
 'to': 8974,
 'amp': 738,
 'families': 3013,
 'in': 3780,
 'asapnaans': 1084,
 'doesn': 2699,
 'matter': 4530,
 'how': 3644,
 'many': 4492,
 'vine': 9239,
 'they': 8898,
 're': 6605,
 'always': 658,
 'funny': 3248,
 'drwwqvkitn': 2773,

In [17]:
# is this the missing line of code??

inverse_vocab = {idx: word for word, idx in tfidf_vec.vocabulary_.items()}

In [18]:
from collections import Counter, defaultdict
by_cls = defaultdict(Counter)
for word_idx, pred in enumerate(word_pred):
    for class_idx, score in enumerate(pred):
        cls = label_encoder.classes_[class_idx]
        by_cls[cls][inverse_vocab[word_idx]] = score

In [19]:
for k in by_cls:
    words = [x[0] for x in by_cls[k].most_common(5)]
    print (k, ':', ' '.join(words))

emoji : studying 7c 1845 thalaajith_fc moderato
© : serendipitypjm_ 23 24m continuetour ser
‼ : beautybygeorg living spreads twins sm0ldlkvop
▶ : 2x6rpofguk ps4 qbnquswv1g pnbxvfmkhl 5tesfy6kml
☹ : v_of_europe amazingphil sean imamofpeace offic
☺ : youuu benz boxerdogjoe moms banger
♥ : aaa shanefilanindia jgbvgozbd0 avni partner
✅ : quite 𝐚𝐫𝐞 아이콘 xozco omo_iya_abudu
✌ : 4reedjc3r2 ancy name white battle
✨ : 0bry9p35yk partying 5qdxehmykm sirboring_26 ian
❤ : preetggtt opsle4kovy weeks patriciocora shadow
➡ : 2moons2audition pranks purposes championsleague shhitspablo_
🎃 : jai riotgames sizwe_phungwayo 4250_ballout padre
🎉 : pictures rowanna_09 sinbadbad bbma yeri
🎶 : 3zutzlltoi 8th result spurs dramatic
🏆 : bfqx9iskqx pakistanzindabad wont hnkmvg5dgo primeministerimrankhan
👀 : bts_army qhrkel1s7b couldn notification pokimanelol
👇 : cringe above robert rajasthan talented
👉 : hip aj5inj3fll tn ricky hilarious
👌 : evidence nddw carpet yuvination bts_bighit
👍 : dreams along raclette great